In [25]:
import numpy as np
import cs_vqe as c

In [26]:
ham = {'III':-6.823060333, 'ZII':-0.1110098029, 'IZI':-0.5370907285, 'IIZ':-0.3127149146, 'ZZI':0.383637914, 'ZIZ':0.2581256772, 'IZZ':0.2523178271, 'XXI':0.06593809513, 'YYI':-0.06593809513, 'XIX':0.0121680127, 'YIY':0.0121680127, 'IXX':0.01764480014, 'IYY':0.01764480014}

In [27]:
c.contextualQ_ham(ham)

True

In [28]:
c.greedy_dfs(ham, 10, criterion='weight')

[[], ['III', 'IZI', 'ZZI', 'IIZ', 'ZIZ', 'IZZ', 'ZII', 'XXI', 'YYI']]

In [29]:
terms_noncon = {'III', 'ZII', 'IZI', 'IIZ', 'ZZI', 'ZIZ', 'IZZ', 'IXX', 'IYY'}
ham_noncon = {p:ham[p] for p in terms_noncon}
model = c.quasi_model(ham_noncon)
#model

In [30]:
c.contextualQ_ham(ham_noncon)

False

In [31]:
print('commuting generators:',model[0], '\n')
print('anticommuting generators:',model[1], '\n')
print('term reconstruction:',model[2], '\n')

commuting generators: ['ZII', 'IZZ'] 

anticommuting generators: ['IXX', 'IIZ'] 

term reconstruction: {'IYY': [['IZZ'], ['IXX'], (-1+0j)], 'IZI': [['IZZ'], ['IIZ'], 1], 'ZIZ': [['ZII'], ['IIZ'], 1], 'III': [[], [], 1], 'IXX': [[], ['IXX'], 1], 'ZII': [['ZII'], [], 1], 'ZZI': [['ZII', 'IZZ'], ['IIZ'], 1], 'IIZ': [[], ['IIZ'], 1], 'IZZ': [['IZZ'], [], 1]} 



In [32]:
fn_form = c.energy_function_form(ham_noncon, model)
print('dim of q: ', fn_form[0])
print('dim of r: ', fn_form[1])
print('encoding of function: ', fn_form[2], '\n')

dim of q:  2
dim of r:  2
encoding of function:  [[(-0.01764480014+0j), [1], [0], 'IYY'], [-0.5370907285, [1], [1], 'IZI'], [0.2581256772, [0], [1], 'ZIZ'], [-6.823060333, [], [], 'III'], [0.01764480014, [], [0], 'IXX'], [-0.1110098029, [0], [], 'ZII'], [0.383637914, [0, 1], [1], 'ZZI'], [-0.3127149146, [], [1], 'IIZ'], [0.2523178271, [1], [], 'IZZ']] 



In [33]:
gs_noncon = c.find_gs_noncon(ham_noncon)
print('ground state energy estimate:', gs_noncon[0], '\n')
print('energy-minimizing parameter setting:', gs_noncon[1])
ep_state = gs_noncon[1]

ground state energy estimate: -7.951301937300001 

energy-minimizing parameter setting: [[-1, 1], [2.3209029648311383e-08, 0.9999999999999998]]


In [34]:
c.quantum_correction(ham, model, fn_form, ep_state)

-7.951301937300001

In [51]:
order = [2,1,0]
reduced_hamiltonians = c.get_reduced_hamiltonians(ham, model, fn_form, ep_state, order)

print('pure noncontextual approximation:',reduced_hamiltonians[0],'\n')

for i in range(1, fn_form[0]+fn_form[1]):
    print('using %i qubits on quantum processor:' % i, reduced_hamiltonians[i], '\n')
    
q3 = reduced_hamiltonians[3] 

pure noncontextual approximation: {'': -7.951301937300001} 

using 1 qubits on quantum processor: {'I': -6.459732703, 'X': -3.4617874580971315e-08, 'Z': -1.4915692342999998} 

using 2 qubits on quantum processor: {'II': -6.7120505301, 'IX': 0.017644778770781636, 'ZZ': 0.9207286429095184, 'ZX': 0.017644813388656217, 'IZ': -0.5708405913904813, 'ZI': -0.2523178271} 

using 3 qubits on quantum processor: {'III': -6.823060333, 'ZII': -0.1110098029, 'IIX': 0.017644787674645356, 'IZZ': 0.5370907289095185, 'IZX': 0.01764480739780972, 'IIZ': -0.31271491419048125, 'ZIX': 8.903863720242333e-09, 'ZZZ': -0.3836379139999999, 'ZZX': -5.990846495125254e-09, 'ZIZ': 0.25812567719999996, 'IZI': -0.2523178271, 'XXZ': -0.06593809513, 'YXY': -1.5303592048253463e-09, 'YYI': -0.06593809512999999, 'XXI': -2.8240776751532946e-10, 'XYY': -0.012168012699999996, 'YYX': 0.0121680127} 



In [61]:
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators.legacy import WeightedPauliOperator

ham_P = []

for p in q3:
    P = Pauli(label = str(p))
    ham_P.append([q3[p], P])

ham_P = WeightedPauliOperator(ham_P)
num_particles = 2
num_spin_orbitals = 6
shift=-6.609784771130335

In [62]:
backend = BasicAer.get_backend("statevector_simulator")
#distances = np.arange(0.5, 4.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)

qubitOp = ham_P
result = NumPyEigensolver(ham_P).run()
print(result)

exact_energies.append(np.real(result.eigenvalues) + shift)
initial_state = HartreeFock(
    num_spin_orbitals,
    num_particles,
    qubit_mapping='parity'
) 
var_form = UCCSD(
    num_orbitals=num_spin_orbitals,
    num_particles=num_particles,
    initial_state=initial_state,
    qubit_mapping='parity'
)
vqe = VQE(qubitOp, var_form, optimizer)
vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
vqe_energies.append(vqe_result)
print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])

print("All energies have been calculated")


{'eigenvalues': array([-7.95219971+1.03207409e-16j]), 'eigenstates': ListOp([VectorStateFn(Statevector([-9.57392408e-04+1.16066514e-03j,
              2.34582942e-02-2.84389390e-02j,
             -1.11022302e-16+6.66133815e-16j,
             -1.31838984e-15+3.16413562e-15j,
             -4.21884749e-15+0.00000000e+00j,
             -4.37150316e-16+6.10622664e-16j,
              6.35887015e-01-7.70897997e-01j,
             -1.03670606e-03+1.25681859e-03j],
            dims=(2, 2, 2)), coeff=1.0, is_measurement=False)], coeff=1.0, abelian=False)}


C:\Users\Tim\Anaconda3\envs\Qiskit\lib\site-packages\qiskit\chemistry\components\initial_states\hartree_fock.py:61: DeprecationWarning: The HartreeFock class is deprecated as of Aqua 0.9 and will be removed no earlier than 3 months after the release date. Instead, all algorithms and circuits accept a plain QuantumCircuit. 
  super().__init__()
C:\Users\Tim\Anaconda3\envs\Qiskit\lib\site-packages\qiskit\aqua\components\variational_forms\variational_form.py:48: DeprecationWarning: The package qiskit.aqua.components.variational_forms is deprecated. For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('aqua.components.variational_forms')
C:\Users\Tim\Anaconda3\envs\Qiskit\lib\site-packages\qiskit\chemistry\components\variational_forms\uccsd.py:248: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.algorithm_globals (pip install qiskit-terra). For more information 

AquaError: 'The number of qubits of the variational form does not match the operator, and the variational form does not allow setting the number of qubits using `num_qubits`.'

In [56]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")

AttributeError: 'dict' object has no attribute 'num_qubits'